In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pyproj import CRS
from shapely import geometry, ops

In [2]:
network2 = gpd.read_file("C:/data/L3Pilot/katuverkko/Turku/Shape/TurkuKVL.shp") 
#network2 = gpd.read_file("C:/data/L3Pilot/katuverkko/Helsinki/hki_link_liikennemääräkartta2019/test/Kehä1.shp")

In [3]:
collist = list(network2.columns)

In [4]:
#network = gpd.GeoDataFrame(network2, columns=collist)

In [5]:
network2["KATULUOKKA"] = network2["toimluok_l"]

In [6]:
#Turun KVL-aineistossa on Nonetype kadunnimiä. Korvataan:
katulist = list(network2["katu"])
nonelist = [1]
otherlist = [str(1)]

for å in range(0, len(network2)):
    if network2["katu"].iloc[å] == None:
        nonelist.append(1)
        otherlist.append(str(sum(nonelist)))

nimetonlist = []
for w in range(0, len(otherlist)):
    nimetonlist.append("nimeton"+otherlist[w])

orderlist = [1]
for e in range(0, len(katulist)):
    if katulist[e] == None:
        katulist[e] = nimetonlist[sum(orderlist)-1]
        orderlist.append(1)
                         

In [7]:
network2["katu"] = gpd.GeoSeries(katulist)

C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  """Entry point for launching an IPython kernel.


In [8]:
grouped = network2.groupby("katu")

In [9]:
#alternative katulist
katulist = list(network2["katu"])
my_set = set(katulist)
uniquekadut = list(my_set)

In [10]:
len(uniquekadut)

321

In [71]:
%%capture

testilista = []
homogeeniset_linkit = gpd.GeoDataFrame()
mergetest = gpd.GeoDataFrame()

#for i in katulist: #Iterate over streets
for i in uniquekadut:
    grp = grouped.get_group(i)
    KLgrouped = grp.groupby("toimluok_l") #Group by road class
    
    #Iterate over classes in street
    for key, y in KLgrouped:
        KLgrp = KLgrouped.get_group(key)
        #Dissolve road sections that are of the same class in the same street
        dissolved = KLgrp.dissolve(by="toimluok_l") 
        dissolved = dissolved.reset_index()
        
        #Select sections which are not multilinestring. They can be added to main GDF.
        if str(type(dissolved.iloc[0, 1])) != "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
            homogeeniset_linkit = homogeeniset_linkit.append(dissolved.iloc[0])
        
        #Select sections constisting of multilinestrings. These secitons are not next to each other, and must be made separate for the main GDF
        elif str(type(dissolved.iloc[0, 1])) == "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
            merge = ops.linemerge(dissolved.iloc[0, 1]) #now have the n. of continuous pieces as multilinestring object. Must be merged to join otherwise separate but connected lines.
            
            #Select sections merged to multilinestrings
            if str(type(merge)) == "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
                dissolved3 = gpd.GeoDataFrame() #Create GDF to which we will collect these as seperate sections
                bufdf = gpd.GeoDataFrame() #Create GDF to which we will collect buffers of the seperate sections
                buflist = []
                for b in range(0, (len(merge))): #Create enough rows to accommodate all sections
                    dissolved3 = dissolved3.append(dissolved.iloc[0])
                
                #Replace old geometry with seperate sections
                for x in range(0, len(dissolved3)): 
                    dissolved3["geometry"].iloc[x] = merge[x]
                    
                dissolved3 = dissolved3.reset_index()
 
                #BUFFERS --------- Buffers must be created around the seperate sections selected above. 
                #This is due to links in Helsinki network not always being spatially connected in the shape file (difference of 0.1-2m usually, 
                #even though they represent roads connected in reality. Buffers are used to bridge these links.
                
                #Create buffers for all seperated links
                for z in range(0, len(dissolved3)): 
                    buf = dissolved3["geometry"].iloc[z].buffer(8) #---------ADJUST BUFFER SIZE HERE-------------
                    buflist.append(buf) #Add buffers to a list
                
                dissolved3 = dissolved3.drop(["index"], axis="columns") #Remove index column from dissolved3
                bufdf["geometry"] = gpd.GeoSeries(buflist) #Add contents of buflist to bufdf 
                bufdf.reset_index()
                
                #Create empty geodataframes to be filled with ID concerning whether buffers touch, and how they touch other buffers
                touchingbufs = gpd.GeoDataFrame()
                maxidissolve = gpd.GeoDataFrame()
                
                touchingbufslist = [] #Here we will collect the information concerning what buffers a buffer touches
                excesslist = [] 
                
                #Iterate over buffers for separated links. Add buffers to "touchingbufs" and a corresponding ID-value to touchingbufslist.
                for v in list(range(0, len(bufdf)-1)):
                    
                    # FIRST, check if first feature touches another buffer or stands alone. Append touchingbufslist accordingly
                    if len(touchingbufslist) == 0:
                        if (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"])): #Collect intersecting buffers into one df
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("s") #Create ID to identify continuous section
                        elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"]) == False):
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("own") #Append ID
                    #front and back        
                    elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"])) and (bufdf.loc[v]["geometry"].intersects(bufdf.loc[int(v-1)]["geometry"])): 
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("fb") #Append ID
                    #back only        
                    elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v-1]["geometry"])) and ((bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"])) == False): 
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("b") #Append ID
                    #front only        
                    elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v-1]["geometry"]) == False) and ((bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"]))): 
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("s") #Append ID
                    #unconnected        
                    elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v-1]["geometry"]) == False) and ((bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"])) == False): 
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("own") #Append ID
                    #excess        
                    elif KeyError:
                        excesslist.append(1)


                touchingbufs = touchingbufs.reset_index()
                touchingbufs["startstop"] = gpd.GeoSeries(touchingbufslist) #Add intersecting buffer ID to GDF which contains the buffers
                
                #Give ID-Values to buffers. These will be used to dissolve continuous sections .
                slist = []
                idlist = []
                for k in range(0, len(touchingbufs)):
                    if touchingbufs["startstop"].iloc[k] == "s":
                        slist.append(1)
                        idlist.append(sum(slist))
                    elif touchingbufs["startstop"].iloc[k] == "fb":
                        idlist.append(sum(slist))
                    elif touchingbufs["startstop"].iloc[k] == "b":
                        idlist.append(sum(slist))
                        #slist.append(1)
                    elif touchingbufs["startstop"].iloc[k] == "own":
                        slist.append(1)
                        idlist.append(sum(slist))
                        test = sum(slist)
                
                if len(touchingbufs) > 0:
                    touchingbufs["dissolve"] = gpd.GeoDataFrame(idlist)
                    touchingbufs["BufID"] = gpd.GeoDataFrame(idlist)
                    touchingbufs["BufID2"] = gpd.GeoDataFrame(idlist)
                    
                    tdissolve = touchingbufs.dissolve(by="dissolve") #Dissolve buffers that touch by the intersecting buffer ID
                if len(tdissolve) > 0: #If tdissolve contains something, add it to maxidissolve GDF
                    maxidissolve = maxidissolve.append(tdissolve)
                               
                # Join with previously merged sections
                bufjoin = gpd.sjoin(dissolved3, maxidissolve, how="left", op="within") #Spatially join links (lines) with buffers
                bufjoin = bufjoin.reset_index()
                bufjoinfinal = bufjoin.dissolve(by="BufID") #Dissolve by BufferID
                bufjoinfinal = bufjoinfinal.reset_index()
                

                for n in range(0, len(bufjoin)): #Add links that were not within buffers
                    if pd.isna(bufjoin["BufID2"].loc[n]):
                        bufjoinfinal = bufjoinfinal.append(bufjoin.iloc[n])
                
                for q in range(0, len(bufjoinfinal)):
                    homogeeniset_linkit = homogeeniset_linkit.append(bufjoinfinal.iloc[q]) # append final links to main GDF

            else: # Any other sections need no attention and can be passed to the main GDF
                dissolved4 = gpd.GeoDataFrame()
                dissolved4 = dissolved4.append(dissolved.iloc[0])
                dissolved4["geometry"].iloc[0] = merge
                homogeeniset_linkit = homogeeniset_linkit.append(dissolved4.iloc[0])
        

homogeeniset_linkit = homogeeniset_linkit.drop(["level_0"], axis="columns")
homogeeniset_linkit = homogeeniset_linkit.reset_index()
homogeeniset_linkit.crs = CRS.from_epsg(3067)

In [72]:
bufjoinfinal

,BufID,geometry,level_0,ID,ID_NEW,ID_old,KATULUOKKA,Lajin_seli,aht_ajon,apu,...,uusin_lask,uusin_ra_1,uusin_rask,vrkjak07_1,vrkjak19_2,vrkjak22_0,index_right,index,startstop,BufID2
0,1.0,"LINESTRING (239367.971 6710917.509, 239358.785...",0,70.0,7070.0,451.0,0.0,"yaoh#katu,keskilinja",0.0,2003.0,...,2003.0,2003.0,60.0,79.0,13.0,8.0,1.0,0.0,own,1.0
1,2.0,"LINESTRING (240057.968 6711326.131, 239923.619...",1,70.0,7070.0,451.0,0.0,"yaoh#katu,keskilinja",0.0,2003.0,...,2003.0,2003.0,60.0,79.0,13.0,8.0,2.0,1.0,s,2.0
2,NaN,"LINESTRING (240274.146 6711452.723, 240269.658...",2,70.0,7070.0,451.0,0.0,"yaoh#katu,keskilinja",0.0,2003.0,...,2003.0,2003.0,60.0,79.0,13.0,8.0,NaN,NaN,NaN,NaN


In [73]:
homogeeniset_linkit = homogeeniset_linkit[["KATULUOKKA", "geometry", "katu"]]

In [74]:
homogeeniset_linkit.head()
homogeeniset_linkit["H_ID"] = list(range(0, len(homogeeniset_linkit))) # Create Link identifier

In [75]:
join = gpd.sjoin(homogeeniset_linkit, network2, how="left", op="contains")

In [76]:
katulist = list(join["katu_left"])

H_ID = list(join["H_ID"])

katuIDdf = gpd.GeoDataFrame()
katuIDdf["katu"] = gpd.GeoSeries(katulist)
katuIDdf["H_ID"] = gpd.GeoSeries(H_ID)

C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:6: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  
C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:7: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  import sys


In [77]:
finaldissolve1 = join.dissolve(by='H_ID', aggfunc='mean')

In [78]:
NADF = gpd.GeoDataFrame()



fdlen = len(finaldissolve1)

#for x in range(0, fdlen):
    #if pd.isna(finaldissolve1["SY KAVL+RV"].iloc[x]):
        #finaldissolve1 = finaldissolve1.drop(finaldissolve1.index[x])     
        
for x in range(0, len(finaldissolve1)):
    if pd.isna(finaldissolve1["kavl_NYKY"].iloc[x]):
        row = finaldissolve1.iloc[x]
        NADF = NADF.append(finaldissolve1.iloc[x])
        

        
#NADF = NADF.drop(["index_right"], axis="columns")
NADF["H_ID"] = list(range(0, len(NADF))) # Create Link identifier
NADF.crs = CRS.from_epsg(3067)

NADFIND = list(NADF.index)
#KLID = list(katuIDdf["KL"])
KL_LIST = list(NADF["KATULUOKKA_right"])



katulist2 = []

for a in range(0, len(NADFIND)):
    for b in range(0, len(katuIDdf)):
        if int(NADFIND[a]) == int(katuIDdf["H_ID"].iloc[b]):
            katulist2.append(str(katuIDdf["katu"].iloc[b]))

In [79]:
NADF = NADF.drop(["index_right"], axis="columns")

In [80]:
NADFIND_FINALDISSOLVE1 = list(finaldissolve1.index)
katulist3 = []

for c in range(0, len(NADFIND_FINALDISSOLVE1)):
    for d in range(0, len(homogeeniset_linkit)):
        if int(NADFIND_FINALDISSOLVE1[c]) == int(homogeeniset_linkit["H_ID"].iloc[d]):
            katulist3.append(str(homogeeniset_linkit["katu"].iloc[d]))

In [81]:
finaldissolve1 = finaldissolve1.dropna(subset=['kavl_NYKY'], axis='rows')

In [82]:
finaldissolve1["katu"] = gpd.GeoDataFrame(katulist3)

In [83]:
len(NADF)

34

In [84]:
nulljoin = gpd.sjoin(NADF, network2, how="left", op="intersects")

In [85]:
nulldissolve = nulljoin.dissolve(by='H_ID', aggfunc='mean')

In [86]:
nulldissolve = nulldissolve.drop([
'ID_left', 'ID_NEW_left', 'ID_old_left', 'KATULUOKKA_left', 'KATULUOKKA_right', 'aht_ajon_left', 'apu_left', 'apuraskaat_left',
       'iht_ajon_left', 'kavl_2008_left', 'kavl_2011_left', 'kavl_2017_left',
       'kavl_2018_left', 'kavl_NYKY_left', 'keskiviiva_left', 'luotettavu_left', 'nopeus_left', 'pituus_left',
       'rprosentti_left', 'toimluok_l_left', 'uusin_kavl_left',
       'uusin_lask_left', 'uusin_ra_1_left', 'uusin_rask_left',
       'vrkjak07_1_left', 'vrkjak19_2_left', 'vrkjak22_0_left', 'index_right',
       'ID_old_right', 'toimluok_l_right', 'luotettavu_right', 'kavl_2018_right', 'kavl_2017_right', 'kavl_2011_right',
       'kavl_2008_right', 'uusin_kavl_right', 'uusin_lask_right',
       'uusin_rask_right', 'uusin_ra_1_right', 'aht_ajon_right',
       'iht_ajon_right', 'nopeus_right', 'rprosentti_right',
       'vrkjak07_1_right', 'vrkjak19_2_right', 'vrkjak22_0_right',
       'pituus_right', 'keskiviiva_right', 'apu_right', 'apuraskaat_right',
       'ID_right', 'ID_NEW_right', ], axis="columns")

In [87]:
nulldissolve.head()

,geometry,kavl_NYKY_right,KATULUOKKA
H_ID,,,
0,"LINESTRING (237454.434 6709557.360, 237453.170...",2100.0,5.0
1,"LINESTRING (237554.477 6709644.304, 237545.900...",2100.0,5.0
2,"LINESTRING (237986.578 6707641.989, 237933.943...",2000.0,0.0
3,"MULTILINESTRING ((238368.125 6707090.811, 2383...",2000.0,0.0
4,"LINESTRING (238785.633 6708151.784, 238763.905...",1400.0,0.0


In [88]:
nulldissolve["finaldissolve1LUOKKA"] = gpd.GeoSeries(KL_LIST)
nulldissolve["katu"] = gpd.GeoSeries(katulist2)

C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  """Entry point for launching an IPython kernel.
C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  


In [89]:
finaldissolve1 = finaldissolve1[['geometry', 'katu', 'KATULUOKKA_left', 'kavl_NYKY']]

In [90]:
nulldissolve.columns

Index(['geometry', 'kavl_NYKY_right', 'KATULUOKKA', 'finaldissolve1LUOKKA',
       'katu'],
      dtype='object')

In [91]:
nulldissolve = nulldissolve[['geometry', 'kavl_NYKY_right', 'katu', 'KATULUOKKA']]

In [92]:
finalproduct = finaldissolve1
finalproduct = finalproduct.append(nulldissolve)
finalproduct = finalproduct.reset_index()

In [93]:
finalproduct["ID"] = gpd.GeoSeries(list(range(0, len(finalproduct))))

C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  """Entry point for launching an IPython kernel.


In [94]:
finalproduct["pituus"] = finalproduct.geometry.length

In [95]:
finalproduct.crs = CRS.from_epsg(3067)

In [96]:
homogeeniset_linkit.crs = CRS.from_epsg(3067)

In [97]:
finaldissolve1.crs = CRS.from_epsg(3067)

In [98]:
nulldissolve.crs = CRS.from_epsg(3067)

In [99]:
finalproduct.to_file("C:/data/L3Pilot/katuverkko/Turku/TURKUfinalproduct_contains.shp")

In [ ]:
#finalproduct.to_csv("C:/data/L3Pilot/katuverkko/Helsinki/hki_link_liikennemääräkartta2019/finalproduct.csv")

In [ ]:
#nulldissolve.to_csv("C:/data/L3Pilot/katuverkko/Helsinki/hki_link_liikennemääräkartta2019/nulldissolve.csv")

In [ ]:
#finaldissolve1.to_file("C:/data/L3Pilot/katuverkko/Helsinki/hki_link_liikennemääräkartta2019/finaldissolve.shp")

In [ ]:
#finaldissolve1.to_csv("C:/data/L3Pilot/katuverkko/Helsinki/hki_link_liikennemääräkartta2019/finaldissolve.csv")

In [ ]:
#nulldissolve.to_file("C:/data/L3Pilot/katuverkko/Helsinki/hki_link_liikennemääräkartta2019/nulldissolve.shp")

In [ ]:
#homogeeniset_linkit.to_file("C:/data/L3Pilot/katuverkko/Helsinki/hki_link_liikennemääräkartta2019/homogeeniset_linkit_mergetest_connectionsfixed.shp")

In [ ]:
#homogeeniset_linkit.to_csv("C:/data/L3Pilot/katuverkko/Helsinki/hki_link_liikennemääräkartta2019/homogeeniset_linkit_mergetest_connectionsfixed.csv")